# Parallel Chains
- A parallel chain means doing many tasks at the same time instead of one after another.

Think of it like this:

- Sequential chain = You make coffee ☕, then toast bread 🍞, then fry an egg 🍳 (one by one).

- Parallel chain = You make coffee, toast bread, and fry an egg all at once ⏱️ → breakfast is ready faster!

### In LangChain:

- You send the same input (like text) to multiple models/tasks at once.
- Each task works independently and gives its own result.
- Finally, you can combine all those results into one output.



In [2]:
! pip install langchain-google-genai

In [26]:
# Importing Necessary Libraries
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel
from google.colab import userdata


In [4]:
# Getting the API KEY
API_KEY = userdata.get('GOOGLE_API_KEY')


In [5]:
# Choosing the model
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    google_api_key = API_KEY
)


In [27]:
# Providing 2 templates
template1 = [
    ("system", "You are a movie critic."),
    ("human", "Provide a brief summary of the movie {movie_name}."),
]

# Converting the template1 into a ChatPromptTemplate
summary_template = ChatPromptTemplate.from_messages(template1)


In [44]:
# Function to analyze plot in a movie
def analyze_plot(plot):
  plot_template = ChatPromptTemplate.from_messages(
      [
          ("system", "You are a movie critic."),
          ("human", "Analyze the plot: {plot}. What are its strengths and weaknesses?"),
      ]
  )
  # Fill the template with actual plot details
  return plot_template.format_prompt(plot =  plot)

# Function to analyze characters in a movie
def analyze_characters(characters):
  character_template = ChatPromptTemplate.from_messages(
      [
           ("system", "You are a movie critic."),
           ("human", "Analyze the characters: {characters}. What are their strengths and weaknesses?"),
      ]
  )
  # Fill the template with actual character names/details
  return character_template.format_prompt(characters = characters)


In [45]:
# Function to combine both plot and character analysis into one final verdict
def combine(analyze_plot, analyze_characters):
  return f"Plot Analysis: \n{analyze_plot}\n\n Character Analysis: \n{analyze_characters}"


In [46]:
# Branch 1: Analyze the plot
plot_branch = (RunnableLambda(lambda x: analyze_plot(x)) | llm | StrOutputParser())

# Branch 2: Analyze the characters
characters_branch = (RunnableLambda(lambda x: analyze_characters(x)) | llm | StrOutputParser())


In [47]:
# Making a chain
chain = (summary_template | llm | StrOutputParser() |
# Running tasks in parallel (faster)
RunnableParallel(branches = {"plot": plot_branch, "characters": characters_branch}) |
# Finally → Combine both results
RunnableLambda(lambda x: combine(x["branches"]["plot"], x["branches"]["characters"]))
)

# Filling the chain with actual values
response = chain.invoke({
    "movie_name": "Inception"
})

# Getting response
print(response)

Plot Analysis: 
Okay, let's delve into the strengths and weaknesses of the plot of *Inception*.

**Strengths:**

*   **High Concept and Originality:** The core concept of stealing or planting ideas through dreams is incredibly compelling and original. It immediately grabs the audience's attention and sets the stage for a unique cinematic experience. It's a fresh take on the heist genre.

*   **Intricate World-Building:** Nolan meticulously crafts the rules and mechanics of the dream world, including dream levels, shared dreaming, projections, and the concept of "limbo." This detailed world-building creates a believable (within the context of the film) and immersive experience. The rules are generally consistent, which is crucial for maintaining the audience's engagement.

*   **Multi-Layered Narrative:** The plot operates on multiple levels, both within the dream layers and on a thematic level. We have the heist itself, Cobb's personal struggle with his deceased wife Mal, and the explo